In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable as V
import torch.nn.functional as F
from  sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import pandas as pd
from torch.utils.data import DataLoader,Dataset,TensorDataset
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
import numpy
from sklearn.model_selection import train_test_split
from collections import OrderedDict

In [75]:
# load in datasets
scnv_512=pd.read_csv("/Users/vivit/Capstone/Glioma/512data_scnv_G.csv")
scnv_256=pd.read_csv("/Users/vivit/Capstone/Glioma/256data_scnv_G.csv")
scnv_128=pd.read_csv("/Users/vivit/Capstone/Glioma/128data_scnv_G.csv")
train_clinical=pd.read_csv("/Users/vivit/Capstone/train_clinical_G.csv")
test_clinical=pd.read_csv("/Users/vivit/Capstone/test_clinical_G.csv")

In [76]:
scnv_512

,Unnamed: 0,1p36.21,1q22,1q32.1,1q44,2p24.3,3q26.33,4p16.3,4q12,5p15.33,...,0.430,0.431,0.432,0.433,0.434,0.435,0.436,0.437,0.438,0.439
0,TCGA.CS.4938,0.0037,0.0031,0.0031,0.0031,0.0032,0.0012,-0.0051,-0.0051,-0.0020,...,0,0,0,0,0,0,0,0,0,0
1,TCGA.CS.4941,0.4585,0.4470,0.4439,0.4439,-0.0091,0.0178,-0.0109,-0.0162,0.0017,...,0,0,0,0,0,0,0,0,0,0
2,TCGA.CS.4942,0.0018,0.0018,0.0018,0.0018,-0.0061,-0.0086,-0.0208,-0.0208,-0.0216,...,0,0,0,0,0,0,0,0,0,0
3,TCGA.CS.4943,-0.0425,-0.0425,-0.0425,-0.0425,-0.0329,-0.0079,0.0784,3.5197,-0.5129,...,0,0,0,0,0,0,0,0,0,0
4,TCGA.CS.4944,-0.0089,0.0000,0.0000,0.0000,0.0008,0.0150,0.0171,0.0171,0.0078,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,TCGA.WY.A85A,-0.0075,-0.0075,-0.0075,-0.0055,-0.0001,0.0202,0.0136,0.0136,0.0061,...,0,0,0,0,0,0,0,0,0,0
504,TCGA.WY.A85B,-0.0088,0.0111,-0.0228,-0.0228,-0.0294,0.0265,0.0000,0.0000,-0.0986,...,0,0,0,0,0,0,0,0,0,0
505,TCGA.WY.A85C,0.0154,0.0154,0.0103,0.0385,0.0022,0.0522,-0.0031,-0.0168,-0.0113,...,0,0,0,0,0,0,0,0,0,0
506,TCGA.WY.A85D,0.0210,0.0013,0.0013,0.0013,0.0000,0.0011,-0.0142,-0.0142,-0.0021,...,0,0,0,0,0,0,0,0,0,0


In [77]:
train_clinical

,Unnamed: 0,Unnamed: 0.1,years_to_birth,histological_type,gender,radiation_therapy,race,ethnicity,overall_survival,status,overallsurvival
0,474,TCGA.TQ.A7RJ,25.0,oligoastrocytoma,female,yes,white,hispanicorlatino,1229.0,0.0,"1229,0"
1,227,TCGA.HT.7468,30.0,oligodendroglioma,male,no,white,nothispanicorlatino,203.0,0.0,"203,0"
2,52,TCGA.DB.A75P,25.0,astrocytoma,female,no,NaN,NaN,492.0,0.0,"492,0"
3,320,TCGA.HT.A619,51.0,oligodendroglioma,female,no,asian,nothispanicorlatino,651.0,0.0,"651,0"
4,221,TCGA.FG.A711,33.0,oligodendroglioma,female,no,white,nothispanicorlatino,1481.0,1.0,"1481,1"
...,...,...,...,...,...,...,...,...,...,...,...
350,98,TCGA.DU.7007,33.0,astrocytoma,male,NaN,white,nothispanicorlatino,1915.0,1.0,"1915,1"
351,476,TCGA.TQ.A7RM,41.0,oligoastrocytoma,female,yes,white,hispanicorlatino,1116.0,0.0,"1116,0"
352,322,TCGA.HT.A61C,66.0,oligodendroglioma,male,yes,white,nothispanicorlatino,537.0,1.0,"537,1"
353,382,TCGA.QH.A6CZ,38.0,oligoastrocytoma,male,no,white,nothispanicorlatino,279.0,0.0,"279,0"


In [78]:
train_idx=train_clinical[['Unnamed: 0.1']]
test_idx=test_clinical[['Unnamed: 0.1']]

In [79]:
train_idx

,Unnamed: 0.1
0,TCGA.TQ.A7RJ
1,TCGA.HT.7468
2,TCGA.DB.A75P
3,TCGA.HT.A619
4,TCGA.FG.A711
...,...
350,TCGA.DU.7007
351,TCGA.TQ.A7RM
352,TCGA.HT.A61C
353,TCGA.QH.A6CZ


In [80]:
scnv_512

,Unnamed: 0,1p36.21,1q22,1q32.1,1q44,2p24.3,3q26.33,4p16.3,4q12,5p15.33,...,0.430,0.431,0.432,0.433,0.434,0.435,0.436,0.437,0.438,0.439
0,TCGA.CS.4938,0.0037,0.0031,0.0031,0.0031,0.0032,0.0012,-0.0051,-0.0051,-0.0020,...,0,0,0,0,0,0,0,0,0,0
1,TCGA.CS.4941,0.4585,0.4470,0.4439,0.4439,-0.0091,0.0178,-0.0109,-0.0162,0.0017,...,0,0,0,0,0,0,0,0,0,0
2,TCGA.CS.4942,0.0018,0.0018,0.0018,0.0018,-0.0061,-0.0086,-0.0208,-0.0208,-0.0216,...,0,0,0,0,0,0,0,0,0,0
3,TCGA.CS.4943,-0.0425,-0.0425,-0.0425,-0.0425,-0.0329,-0.0079,0.0784,3.5197,-0.5129,...,0,0,0,0,0,0,0,0,0,0
4,TCGA.CS.4944,-0.0089,0.0000,0.0000,0.0000,0.0008,0.0150,0.0171,0.0171,0.0078,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,TCGA.WY.A85A,-0.0075,-0.0075,-0.0075,-0.0055,-0.0001,0.0202,0.0136,0.0136,0.0061,...,0,0,0,0,0,0,0,0,0,0
504,TCGA.WY.A85B,-0.0088,0.0111,-0.0228,-0.0228,-0.0294,0.0265,0.0000,0.0000,-0.0986,...,0,0,0,0,0,0,0,0,0,0
505,TCGA.WY.A85C,0.0154,0.0154,0.0103,0.0385,0.0022,0.0522,-0.0031,-0.0168,-0.0113,...,0,0,0,0,0,0,0,0,0,0
506,TCGA.WY.A85D,0.0210,0.0013,0.0013,0.0013,0.0000,0.0011,-0.0142,-0.0142,-0.0021,...,0,0,0,0,0,0,0,0,0,0


In [81]:
trainset___512=scnv_512.merge(train_idx,left_on='Unnamed: 0',right_on='Unnamed: 0.1',how="inner")
testset___512 = scnv_512.merge(test_idx,left_on='Unnamed: 0',right_on='Unnamed: 0.1',how="inner")
trainset___256 = scnv_256.merge(train_idx,left_on='Unnamed: 0',right_on='Unnamed: 0.1',how="inner")
testset___256 = scnv_256.merge(test_idx,left_on='Unnamed: 0',right_on='Unnamed: 0.1',how="inner")
trainset___128 = scnv_128.merge(train_idx,left_on='Unnamed: 0',right_on='Unnamed: 0.1',how="inner")
testset___128 = scnv_128.merge(test_idx,left_on='Unnamed: 0',right_on='Unnamed: 0.1',how="inner")

In [82]:
trainset___512

,Unnamed: 0,1p36.21,1q22,1q32.1,1q44,2p24.3,3q26.33,4p16.3,4q12,5p15.33,...,0.431,0.432,0.433,0.434,0.435,0.436,0.437,0.438,0.439,Unnamed: 0.1
0,TCGA.CS.4941,0.4585,0.4470,0.4439,0.4439,-0.0091,0.0178,-0.0109,-0.0162,0.0017,...,0,0,0,0,0,0,0,0,0,TCGA.CS.4941
1,TCGA.CS.4942,0.0018,0.0018,0.0018,0.0018,-0.0061,-0.0086,-0.0208,-0.0208,-0.0216,...,0,0,0,0,0,0,0,0,0,TCGA.CS.4942
2,TCGA.CS.4943,-0.0425,-0.0425,-0.0425,-0.0425,-0.0329,-0.0079,0.0784,3.5197,-0.5129,...,0,0,0,0,0,0,0,0,0,TCGA.CS.4943
3,TCGA.CS.4944,-0.0089,0.0000,0.0000,0.0000,0.0008,0.0150,0.0171,0.0171,0.0078,...,0,0,0,0,0,0,0,0,0,TCGA.CS.4944
4,TCGA.CS.5393,0.0072,0.0072,-0.0015,-0.0015,-0.0065,0.0022,-0.0057,-0.0057,0.0050,...,0,0,0,0,0,0,0,0,0,TCGA.CS.5393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,TCGA.WY.A858,0.0051,0.0051,0.0051,0.0003,0.0010,-0.0307,0.0036,-0.0079,-0.0567,...,0,0,0,0,0,0,0,0,0,TCGA.WY.A858
351,TCGA.WY.A85A,-0.0075,-0.0075,-0.0075,-0.0055,-0.0001,0.0202,0.0136,0.0136,0.0061,...,0,0,0,0,0,0,0,0,0,TCGA.WY.A85A
352,TCGA.WY.A85B,-0.0088,0.0111,-0.0228,-0.0228,-0.0294,0.0265,0.0000,0.0000,-0.0986,...,0,0,0,0,0,0,0,0,0,TCGA.WY.A85B
353,TCGA.WY.A85C,0.0154,0.0154,0.0103,0.0385,0.0022,0.0522,-0.0031,-0.0168,-0.0113,...,0,0,0,0,0,0,0,0,0,TCGA.WY.A85C


In [83]:
trainset__512=trainset___512.drop("Unnamed: 0",axis=1)
testset__512=testset___512.drop("Unnamed: 0",axis=1)
trainset__256=trainset___256.drop("Unnamed: 0",axis=1)
testset__256=testset___256.drop("Unnamed: 0",axis=1)
trainset__128=trainset___128.drop("Unnamed: 0",axis=1)
testset__128=testset___128.drop("Unnamed: 0",axis=1)

In [84]:
trainset__512

,1p36.21,1q22,1q32.1,1q44,2p24.3,3q26.33,4p16.3,4q12,5p15.33,7p11.2,...,0.431,0.432,0.433,0.434,0.435,0.436,0.437,0.438,0.439,Unnamed: 0.1
0,0.4585,0.4470,0.4439,0.4439,-0.0091,0.0178,-0.0109,-0.0162,0.0017,3.6569,...,0,0,0,0,0,0,0,0,0,TCGA.CS.4941
1,0.0018,0.0018,0.0018,0.0018,-0.0061,-0.0086,-0.0208,-0.0208,-0.0216,0.7006,...,0,0,0,0,0,0,0,0,0,TCGA.CS.4942
2,-0.0425,-0.0425,-0.0425,-0.0425,-0.0329,-0.0079,0.0784,3.5197,-0.5129,0.0128,...,0,0,0,0,0,0,0,0,0,TCGA.CS.4943
3,-0.0089,0.0000,0.0000,0.0000,0.0008,0.0150,0.0171,0.0171,0.0078,0.0161,...,0,0,0,0,0,0,0,0,0,TCGA.CS.4944
4,0.0072,0.0072,-0.0015,-0.0015,-0.0065,0.0022,-0.0057,-0.0057,0.0050,1.7734,...,0,0,0,0,0,0,0,0,0,TCGA.CS.5393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,0.0051,0.0051,0.0051,0.0003,0.0010,-0.0307,0.0036,-0.0079,-0.0567,0.0017,...,0,0,0,0,0,0,0,0,0,TCGA.WY.A858
351,-0.0075,-0.0075,-0.0075,-0.0055,-0.0001,0.0202,0.0136,0.0136,0.0061,0.0049,...,0,0,0,0,0,0,0,0,0,TCGA.WY.A85A
352,-0.0088,0.0111,-0.0228,-0.0228,-0.0294,0.0265,0.0000,0.0000,-0.0986,0.0296,...,0,0,0,0,0,0,0,0,0,TCGA.WY.A85B
353,0.0154,0.0154,0.0103,0.0385,0.0022,0.0522,-0.0031,-0.0168,-0.0113,0.0131,...,0,0,0,0,0,0,0,0,0,TCGA.WY.A85C


In [85]:
trainset_512=trainset__512.drop("Unnamed: 0.1",axis=1)
testset_512=testset__512.drop("Unnamed: 0.1",axis=1)
trainset_256=trainset__256.drop("Unnamed: 0.1",axis=1)
testset_256=testset__256.drop("Unnamed: 0.1",axis=1)
trainset_128=trainset__128.drop("Unnamed: 0.1",axis=1)
testset_128=testset__128.drop("Unnamed: 0.1",axis=1)

In [86]:
trainset_512

,1p36.21,1q22,1q32.1,1q44,2p24.3,3q26.33,4p16.3,4q12,5p15.33,7p11.2,...,0.430,0.431,0.432,0.433,0.434,0.435,0.436,0.437,0.438,0.439
0,0.4585,0.4470,0.4439,0.4439,-0.0091,0.0178,-0.0109,-0.0162,0.0017,3.6569,...,0,0,0,0,0,0,0,0,0,0
1,0.0018,0.0018,0.0018,0.0018,-0.0061,-0.0086,-0.0208,-0.0208,-0.0216,0.7006,...,0,0,0,0,0,0,0,0,0,0
2,-0.0425,-0.0425,-0.0425,-0.0425,-0.0329,-0.0079,0.0784,3.5197,-0.5129,0.0128,...,0,0,0,0,0,0,0,0,0,0
3,-0.0089,0.0000,0.0000,0.0000,0.0008,0.0150,0.0171,0.0171,0.0078,0.0161,...,0,0,0,0,0,0,0,0,0,0
4,0.0072,0.0072,-0.0015,-0.0015,-0.0065,0.0022,-0.0057,-0.0057,0.0050,1.7734,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,0.0051,0.0051,0.0051,0.0003,0.0010,-0.0307,0.0036,-0.0079,-0.0567,0.0017,...,0,0,0,0,0,0,0,0,0,0
351,-0.0075,-0.0075,-0.0075,-0.0055,-0.0001,0.0202,0.0136,0.0136,0.0061,0.0049,...,0,0,0,0,0,0,0,0,0,0
352,-0.0088,0.0111,-0.0228,-0.0228,-0.0294,0.0265,0.0000,0.0000,-0.0986,0.0296,...,0,0,0,0,0,0,0,0,0,0
353,0.0154,0.0154,0.0103,0.0385,0.0022,0.0522,-0.0031,-0.0168,-0.0113,0.0131,...,0,0,0,0,0,0,0,0,0,0


In [87]:
testset_512

,1p36.21,1q22,1q32.1,1q44,2p24.3,3q26.33,4p16.3,4q12,5p15.33,7p11.2,...,0.430,0.431,0.432,0.433,0.434,0.435,0.436,0.437,0.438,0.439
0,0.0037,0.0031,0.0031,0.0031,0.0032,0.0012,-0.0051,-0.0051,-0.0020,0.0114,...,0,0,0,0,0,0,0,0,0,0
1,-0.7710,0.0135,0.0135,0.0135,-0.0052,0.0000,-0.0520,-0.0520,-0.0155,0.0504,...,0,0,0,0,0,0,0,0,0,0
2,-0.8350,0.0184,0.0184,0.0184,-0.0168,0.0025,0.0065,0.0081,0.0000,0.1819,...,0,0,0,0,0,0,0,0,0,0
3,-0.0487,0.0261,0.0261,0.0261,-0.1118,-0.0035,1.4591,-0.0084,0.0338,0.7536,...,0,0,0,0,0,0,0,0,0,0
4,0.0013,0.0013,0.0013,-0.0014,0.0000,0.0008,-0.0035,-0.0035,-0.0032,-0.0003,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,0.0271,0.0038,0.0042,0.0042,0.0000,-0.0050,-0.0019,-0.0115,0.5717,-0.0049,...,0,0,0,0,0,0,0,0,0,0
149,-0.0021,-0.0021,-0.0021,-0.0021,-0.0086,0.0000,-0.2468,-0.0277,-0.0188,0.0088,...,0,0,0,0,0,0,0,0,0,0
150,-0.8630,0.0522,0.0147,0.0147,0.0075,-0.0245,0.0309,-0.0344,-0.0073,0.0088,...,0,0,0,0,0,0,0,0,0,0
151,-0.0196,-0.0196,0.0088,0.0088,0.0192,-0.0153,0.0139,0.0139,-0.7574,0.0129,...,0,0,0,0,0,0,0,0,0,0


In [88]:
trainset512=trainset_512.to_numpy()
testset512=testset_512.to_numpy()
trainset256=trainset_256.to_numpy()
testset256=testset_256.to_numpy()
trainset128=trainset_128.to_numpy()
testset128=testset_128.to_numpy()

In [89]:
# standardize inputs
MMScaler=MinMaxScaler()
train_scaler512=MMScaler.fit_transform(trainset512)
test_scaler512=MMScaler.fit_transform(testset512)
train_scaler256=MMScaler.fit_transform(trainset256)
test_scaler256=MMScaler.fit_transform(testset256)
train_scaler128=MMScaler.fit_transform(trainset128)
test_scaler128=MMScaler.fit_transform(testset128)

In [90]:
train_torch512 = torch.FloatTensor(train_scaler512)
train_loader512 = torch.utils.data.DataLoader(train_torch512, batch_size = 16, shuffle=True)
test_torch512 = torch.FloatTensor(test_scaler512)
test_loader512 = torch.utils.data.DataLoader(test_torch512, batch_size = 16, shuffle=False)

train_torch256 = torch.FloatTensor(train_scaler256)
train_loader256 = torch.utils.data.DataLoader(train_torch256, batch_size = 16, shuffle=True)
test_torch256 = torch.FloatTensor(test_scaler256)
test_loader256 = torch.utils.data.DataLoader(test_torch256, batch_size = 16, shuffle=False)

train_torch128 = torch.FloatTensor(train_scaler128)
train_loader128 = torch.utils.data.DataLoader(train_torch128, batch_size = 16, shuffle=True)
test_torch128 = torch.FloatTensor(test_scaler128)
test_loader128 = torch.utils.data.DataLoader(test_torch128, batch_size = 16, shuffle=False)

In [91]:
# define parameters
EPOCH=10
lr=0.001

In [92]:
train_torch512.shape

torch.Size([355, 512])

In [93]:
test_torch512.shape

torch.Size([153, 512])

In [109]:
class Autoencoder512_2(nn.Module):
    def __init__(self):
        super(Autoencoder512_2, self).__init__()
        # encoder
        self.encoder=nn.Sequential(
            nn.Linear(in_features=512, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=128)           
        )
        self.decoder=nn.Sequential(
            nn.Linear(in_features=128, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=512)
        )
 
    def forward(self, x):
        encoded=self.encoder(x)
        decoded=self.decoder(encoded)
        return decoded
    
    
model512_2 = Autoencoder512_2()
if torch.cuda.is_available():
    model512_2.cuda()
    
model512_2.train(mode=True)
# define optimizer
optimizer512_2 = torch.optim.Adam(model512_2.parameters(), lr=lr)
criterion512_2=nn.MSELoss()


In [110]:
# train the datasets
for epoch in range(EPOCH):
    for batch_features in train_torch512:
        output = model512_2(batch_features)
        loss = criterion512_2(output, batch_features)
        optimizer512_2.zero_grad()
        loss.backward()
        optimizer512_2.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1,EPOCH,loss.data.numpy()))

epoch [1/10], loss:0.0024
epoch [2/10], loss:0.0023
epoch [3/10], loss:0.0020
epoch [4/10], loss:0.0020
epoch [5/10], loss:0.0020
epoch [6/10], loss:0.0019
epoch [7/10], loss:0.0019
epoch [8/10], loss:0.0018
epoch [9/10], loss:0.0019
epoch [10/10], loss:0.0018


In [101]:
class Autoencoder512_1(nn.Module):
    def __init__(self):
        super(Autoencoder512_1, self).__init__()
        # encoder
        self.encoder=nn.Sequential(
            nn.Linear(in_features=512, out_features=256),
            nn.ReLU()
        )
        self.decoder=nn.Sequential(
            nn.Linear(in_features=256, out_features=512),
            nn.ReLU()
        )
 
    def forward(self, x):
        encoded=self.encoder(x)
        decoded=self.decoder(encoded)
        return decoded
    
    
model512_1 = Autoencoder512_1()
if torch.cuda.is_available():
    model512_1.cuda()
    
model512_1.train(mode=True)
# define optimizer
optimizer512_1 = torch.optim.Adam(model512_1.parameters(), lr=lr)
criterion512_1=nn.MSELoss()

In [102]:
# train the datasets
for epoch in range(EPOCH):
    for batch_features in train_torch512:
        output = model512_1(batch_features)
        loss = criterion512_1(output, batch_features)
        optimizer512_1.zero_grad()
        loss.backward()
        optimizer512_1.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1,EPOCH,loss.data.numpy()))

epoch [1/10], loss:0.0107
epoch [2/10], loss:0.0091
epoch [3/10], loss:0.0085
epoch [4/10], loss:0.0065
epoch [5/10], loss:0.0063
epoch [6/10], loss:0.0056
epoch [7/10], loss:0.0048
epoch [8/10], loss:0.0046
epoch [9/10], loss:0.0045
epoch [10/10], loss:0.0044


In [111]:
feature=model512_2.encoder(train_torch512)
feature

tensor([[-0.0996, -0.2795, -0.0404,  ...,  0.0628, -0.1276, -0.0571],
        [-0.0439, -0.1153,  0.1593,  ...,  0.0850,  0.0771, -0.0593],
        [-0.0052, -0.2054, -0.0072,  ..., -0.0079,  0.1186,  0.0121],
        ...,
        [-0.0297, -0.1558,  0.0459,  ...,  0.0480,  0.1441,  0.0372],
        [-0.0364, -0.2099, -0.0016,  ...,  0.0538,  0.1260,  0.0380],
        [-0.0368, -0.2042,  0.0133,  ...,  0.0452,  0.1403,  0.0061]],
       grad_fn=<AddmmBackward>)

In [112]:
a = feature.detach().numpy()
print(a)

[[-0.09958564 -0.27946803 -0.04038219 ...  0.06276756 -0.12757754
  -0.05707984]
 [-0.04394366 -0.11525076  0.1593272  ...  0.08497754  0.07709762
  -0.05931115]
 [-0.00524361 -0.20538606 -0.00715344 ... -0.0078926   0.11862902
   0.01213112]
 ...
 [-0.02971053 -0.15578552  0.04585434 ...  0.04798084  0.14408088
   0.0371738 ]
 [-0.03642812 -0.20990099 -0.00162786 ...  0.05384912  0.12595871
   0.03803581]
 [-0.03683962 -0.20418304  0.01327222 ...  0.04524302  0.14027342
   0.00605841]]


In [120]:
a.shape

(355, 128)

In [121]:
np.savetxt("/Users/vivit/Capstone/SCNV AE results for Glioma Cancer 73.csv",a)

In [116]:
torch.save(model512_2, '/Users/vivit/Capstone/AE model of SCNV for Glioma cancer 73.pkl')

In [98]:
class Autoencoder256(nn.Module):
    def __init__(self):
        super(Autoencoder256, self).__init__()
        # encoder
        self.encoder=nn.Sequential(
            nn.Linear(in_features=256, out_features=128),
            nn.ReLU()
        )
        self.decoder=nn.Sequential(
            nn.Linear(in_features=128, out_features=256),
            nn.ReLU()
        )
 
    def forward(self, x):
        encoded=self.encoder(x)
        decoded=self.decoder(encoded)
        return encoded,decoded
    
    
model256 = Autoencoder256()
if torch.cuda.is_available():
    model256.cuda()
    
model256.train(mode=True)
# define optimizer
optimizer256 = torch.optim.Adam(model256.parameters(), lr=lr)
criterion256=nn.MSELoss()

In [99]:
# train the datasets
for epoch in range(EPOCH):
    for batch_features in train_torch256:
        _,output = model256(batch_features)
        loss = criterion256(output, batch_features)    
        optimizer256.zero_grad()
        loss.backward()
        optimizer256.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1,EPOCH,loss.data.numpy()))

epoch [1/10], loss:0.0203
epoch [2/10], loss:0.0167
epoch [3/10], loss:0.0152
epoch [4/10], loss:0.0148
epoch [5/10], loss:0.0117
epoch [6/10], loss:0.0110
epoch [7/10], loss:0.0109
epoch [8/10], loss:0.0088
epoch [9/10], loss:0.0071
epoch [10/10], loss:0.0069


In [117]:
from sklearn.model_selection import KFold,StratifiedKFold
KF=KFold(n_splits=10,shuffle=True,random_state=123)
print(KF)

KFold(n_splits=10, random_state=123, shuffle=True)


In [118]:
def train_epoch(model,device,dataloader,loss_fun,optimizer):
    train_loss=0.0
    model.train()
    for x in dataloader:
        x=x.to(device)
        optimizer.zero_grad()
        output = model(x)
        loss = loss_fun(output,x)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss=train_loss/len(dataloader)
    return train_loss
def valid_epoch(model,device,dataloader,loss_fun):
    valid_loss = 0.0
    model.eval()
    for x in dataloader:
        x=x.to(device)
        output = model(x)
        loss=loss_func(output,x)
        valid_loss+=loss.item()
    valid_loss+=valid_loss/len(dataloader)
    return valid_loss

In [119]:
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset 
criterion=nn.MSELoss()
K_valid_loss=[]
for fold, (train_idx,val_idx) in enumerate(KF.split(np.arange(len(test_torch512)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = torch.utils.data.DataLoader(train_torch512,batch_size=512,sampler=train_sampler)
    test_loader = torch.utils.data.DataLoader(test_torch512,batch_size=512,sampler=test_sampler)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = Autoencoder512_2()
    model.to(device)
    loss_func=nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    history = {'train_loss': [], 'test_loss': []}
    for epoch in range(EPOCH):
        train_loss=train_epoch(model,device,train_loader,criterion,optimizer)
        test_loss=valid_epoch(model,device,test_loader,criterion)
        print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Test Loss:{:.3f} ".format(epoch + 1,EPOCH,train_loss, test_loss))

Fold 1
Epoch:1/10 AVG Training Loss:0.034 AVG Test Loss:0.065 
Epoch:2/10 AVG Training Loss:0.032 AVG Test Loss:0.061 
Epoch:3/10 AVG Training Loss:0.029 AVG Test Loss:0.055 
Epoch:4/10 AVG Training Loss:0.026 AVG Test Loss:0.048 
Epoch:5/10 AVG Training Loss:0.023 AVG Test Loss:0.040 
Epoch:6/10 AVG Training Loss:0.019 AVG Test Loss:0.031 
Epoch:7/10 AVG Training Loss:0.014 AVG Test Loss:0.024 
Epoch:8/10 AVG Training Loss:0.011 AVG Test Loss:0.021 
Epoch:9/10 AVG Training Loss:0.009 AVG Test Loss:0.019 
Epoch:10/10 AVG Training Loss:0.008 AVG Test Loss:0.016 
Fold 2
Epoch:1/10 AVG Training Loss:0.036 AVG Test Loss:0.072 
Epoch:2/10 AVG Training Loss:0.033 AVG Test Loss:0.068 
Epoch:3/10 AVG Training Loss:0.031 AVG Test Loss:0.063 
Epoch:4/10 AVG Training Loss:0.029 AVG Test Loss:0.058 
Epoch:5/10 AVG Training Loss:0.026 AVG Test Loss:0.051 
Epoch:6/10 AVG Training Loss:0.023 AVG Test Loss:0.044 
Epoch:7/10 AVG Training Loss:0.019 AVG Test Loss:0.036 
Epoch:8/10 AVG Training Loss:0.01